## LINEAR REGRESSION ASSUMPTION (MUTLICOLINEARITY) : VIF AUTOMATED CODE

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices

variables_dropped=[]
x=True
while x:
    ##Creating feature string for using it in VIF formula
    column_str=''

    for col in data_updated.columns.difference(['ln_SalePrice']):
        column_str=column_str+col+'+'


    formula=column_str[:-1]+'~ln_SalePrice'
    X,Y=dmatrices(formula,data_updated,return_type='dataframe')
    print('shape of X variable : ',X.shape)
    print('shape of Y variable : ',Y.shape)
    
    
    ## Calculate VIF
    vif=pd.DataFrame()
    vif['VIF_Factor']=[variance_inflation_factor(X.values,i) for i in range(X.shape[1])]
    vif['Features']=X.columns

    ##Sorting VIF from highest to lowest
    vif.sort_values(by=['VIF_Factor'],ascending=[False],inplace=True)
    vif=vif.reset_index()
    vif.drop('index',axis=1,inplace=True)

    ##Capturing variable for highest VIF
    highest_vif=pd.DataFrame()
    highest_vif=vif.loc[[0],:]
    drop_var=highest_vif.Features[0]

    ##Droppig variable with VIF>10 one by one
    if(highest_vif.VIF_Factor[0]>10):
            print('Variable Dropped : ',drop_var,' as its VIF is : ',highest_vif.VIF_Factor[0])
            print('--------------------------------------')
            variables_dropped.append(drop_var)
            data_updated.drop(drop_var,axis=1,inplace=True)
    else:
            print('All Variables have VIF less than 10')
            x=False
            
print(variables_dropped)

## LINEAR REGRESSION ASSUMPTION  : AUTOMATED CODE FOR CHECKING GRAPHICALLY WHETHER ALL NUMERICAL X VARIABLES ARE LINERARLY RELATED TO Y OR NOT

In [ ]:
for col in data_num.columns.difference(['ln_SalePrice']):
    sns.lmplot(x=col,y='ln_SalePrice',data=data_num,aspect=2)

## CATEGORICAL VARIABLE MISSING VALUE TREATMENT WITH MODE

In [ ]:
## Missing value treatment on Categorical Data

def missing_value_cat(x):
    x=x.fillna(x.mode().iloc[0])
    return x

data_cat=data_cat.apply(lambda x: missing_value_cat(x))

## One hot Encoding for Categorical Variables

In [ ]:
def create_dummies(df,colname):
    col_dummies=pd.get_dummies(df[colname],prefix=colname)
    col_dummies.drop(col_dummies.columns[0],axis=1,inplace=True)
    df=pd.concat([df,col_dummies],axis=1)
    df.drop([colname],axis=1,inplace=True)
    return df


for features in data_cat.columns:
    data_cat[features]=data_cat[features].astype('category')
    data_cat=create_dummies(data_cat,features)
    

    
data_cat.head()

## NUMERICAL VARIABLE OUTILIER TREATMENT 

## NUMERICAL VARIABLE MISSING VALUE TREATMENT WITH MEAN

In [ ]:
## we will do outlier treatment and then missing value treatment

def outlier_clip(x):
    x=x.clip(lower=x.quantile(0.01))
    x=x.clip(upper=x.quantile(0.99))
    return x

data_num=data_num.apply(lambda x: outlier_clip(x))

def missing_value(x):
    x=x.fillna(x.mean())
    x=x.fillna(x.mean())
    return x

data_num=data_num.apply(lambda x: missing_value(x))

data_num.shape

## Create log values of each varible


In [ ]:


for col in data_num.columns:
    data_num['ln_'+col]=np.log(data_num[col]+1)
    
    
data_num.columns

## Creating Model and dropping Variables with p value greater than or equal to 0.05 one - by - one

In [ ]:
variables_dropped_for_pvalue=[]

x=True

while x==True:

    #creating model
    linear_model=sm.OLS(Y_variable,X_variable).fit()
    
    #sorting p values high to low
    pvaules_high_to_low=linear_model.pvalues.sort_values(ascending=False)
    
    #converting variable names as column
    pvaules_high_to_low=pvaules_high_to_low.reset_index()
    
    #renaming columns
    pvaules_high_to_low=pvaules_high_to_low.rename(columns={'index':'Variable_Name',0:'pvalue'})
    
    #CAPTURING HIGHEST PVALUE VARIABLE IN DATAFRAME
    highest_pvalue=pd.DataFrame()
    highest_pvalue=pvaules_high_to_low.loc[[0],:]
    
    ##Checking if pvalue is greater than or equal to 0.05 then droppping variable
    if(highest_pvalue.pvalue[0]>=0.05):
        print(highest_pvalue.Variable_Name[0],' has p _values : ',highest_pvalue.pvalue[0],' which is greater than 0.05')
        variables_dropped_for_pvalue.append(highest_pvalue.Variable_Name[0])
        X_variable.drop(highest_pvalue.Variable_Name[0],axis=1,inplace=True)
        
    else:
        print('\n')
        print('Now All variables have p value less than 0.05')
        x=False

print(X_variable.shape)